# 文本和字节序列

人类可读的文本字符串和字节序列是完全不同的两个事物。

字符串即字符的序列，重点是“字符”如何定义。

对于Python，“字符”即Unicode字符。字符的相关操作涉及到两个问题：字符的标识和字符的字节表述。前者即码位，是特定字符在字符集中的唯一标识。后者则是字符的具体表述，通过特定的编码算法连接字符和对应的字节表述。

字符的编码：将码位转换为字节序列的过程
字符的解码：将字节序列转换为码位的过程

Python对字符和字节进行了完全的区分，具体来说，所有的字节序列的字面量均会以特定标识开头。下述为编/解码的一个示例

In [6]:
test_str = "test！。"
print(test_str)
test_bytes = test_str.encode('utf-8')
print(test_bytes)
test_str_2 = test_bytes.decode('utf-8')
print(test_str_2)

test！。
b'test\xef\xbc\x81\xe3\x80\x82'
test！。


字符串比较关键的问题是字符的编码算法以及文本文件的处理。前者涉及到的问题众多，而且相关的乱码问题时不时会出现，例如matplotlib的中文支持问题，后者则在读写文件时时常会遇到。

## 字节概要

Python 3和Python 2在字符处理上有较大差别，其中比较重要的就是：Python 3引入bytes类型

bytes是一种不可变数据类型，其元素是0~255之间的整数，并且bytes类型数据的切片依然是bytes类型。bytearray类型和bytes类型密切相关，bytearray没有自己的字面量语法，但是和bytes类型的行为一致，其切片依然是bytearray类型。

字节的字面量以类型而定，具体来说，可以分为如下的几类：
1. 可打印的ASCII字节：使用ASCII字符本身
2. 制表符、换行符等：使用独有转义序列
3. 其他字节：使用十六进制转义序列

bytes实例可以使用多种方法构建，其中最常用的显然是通过一个str对象和对应的encoding方法。此外，还可以通过包含0~255之间数值的可迭代对象等方法。

Python提供了一些工具用来处理二进制数据，本书以二进制图像数据的处理为例，展示如何利用struct和memoryview对这些二进制数据进行处理。

## Python中的编解码器以及编码问题

Python自带了相当数量的编解码器，常用的例如utf-8、gbk等，了解下就好，需要的时候再说

编解码问题会导致一些异常的抛出：
1. UnicodeEncodeError：str转bytes时
2. UnicodeDecodeError：bytes转str时
3. SyntaxError：源码的编码错误

### UnicodeEncodeError的处理
通常出现在使用某一特定编码算法编码不支持的字符时，此时可以通过errors参数指定处理方式
1. ignore：忽略无法处理的字符
2. replace：将无法处理的字符替换为?
3. xmlcharrefreplace：将无法处理的字符替换为XML实体
4. 其他方式：自定义处理函数以及名称（通过codecs.register_error）

或者，直接换个编码算法。

### UnicodeDecodeError的处理
UnicodeDecodeError并不总是会在出现无法处理的字节时抛出，一些解码器会处理这些字节而不抛出错误，当然，输出结果自然是没有意义的（乱码）

与UnicodeEncodeError的处理类似，可以通过指定出现无法处理的字节时的处理算法

### SyntaxError的处理

源码编码出现错误时会抛出该异常

若是单纯的字符编码问题，可以在文件顶部使用coding注释来指定编码算法，Python 3默认使用utf-8，Python 2默认使用ASCII，相关内容可参考PEP-263

### 如何辨识编码方式

这个问题很有意思，答案也是很明确的：无法通过字节序列准确辨识编码方式，但是能够通过字节序列的一些特定以及前提条件推断编码方式

推断编码方式需要基于一定的假设：已知待解析的字节流的应用场合。最为广泛的应用场合自然是人类可读的纯文本，因此可以通过字节频次分析（某一特定的字节总是反复出现）等手段判断其对应的编码方式。

Python的Chardet包能够实现上述的功能，其支持识别数十种编码方式

## 处理文本文件

显然，对文本文件的正确处理依赖于使用正确的编解码算法。本章提到，处理文本文件的最佳实践是："Unicode Sandwich"，即文本文件的读取、处理和写入，其分别为文本文件的解码、处理和编码。在处理过程中尽量不要进行编解码。

### 默认编码导致的问题

Python默认使用系统默认的编解码方法。若在写入时指定了encoding参数，但是在读取时没有指定encoding，极有可能导致乱码的出现。

### 编码默认参数

编码默认参数可以通过如下的程序获取

In [3]:
import sys, locale

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open("dummy", "w")

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'cp936'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'cp936'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


上述结果和本章展示的输出结果不一致，推测是由于版本变化导致的，测试操作系统为win10 19044

首先是默认文本文件编码方式locale.getpreferredencoding()，其值为cp936

然后是文本编码方法my_file.encoding，其值也为cp936

由于这里是在文件中进行测试，因此sys.stdout、sys.stdin以及sys.stderr的编码方式均为utf-8

在控制台测试会发现sys.stdout、sys.stdin以及sys.stderr的编码方式也为utf-8，这和本书中的描述不一致

sys.getfilesystemencoding()获取的是文件名的默认编码方式

sys.getdefaultencoding()获得的是Python内部进行二进制数据和字符串转换时使用的编码方法

上述结果在不同的操作系统，甚至同一操作系统的不同版本中可能会有不同的值，因此对于文本文件的处理来说，最佳实践建议是：别依赖默认编码方式

## 字符串规范化

进行字符串之间的比较是比较常见的功能需求。不包含组合字符的字符串比较容易处理。组合字符很可能包含多个不同的表示方法。从应用的角度来说，使用两种方法进行过表示的同一字符应当是等价的，但是从表示的角度来说，由于确实是不同的码位序列，因此简单的逻辑判断会认为二者不相等。

上述问题的解决方案即为字符串规范化，即同一字符的表示方法，然后再进行比较。

unicodedata库提供了normalize方法来实现这一功能。具体而言，通过指定字符的表示形式，将传入的字符串转换为相应的形式。

normalize提供了四种规范化标准：

1. NFC：使用最少码位构成等价字符串
2. NFD：将所有的组合字符解析为基字符和单独的组合字符
3. NFKC：基本上就是NFC的兼容模式，添加了对兼容字符的处理
4. NFKD：基本上就是NFD的兼容模式，添加了对兼容字符的处理

NFKC和NFKD要谨慎使用，其对兼容字符的处理很可能导致语义发生变化

除了组合字符以及兼容字符外，字母的大小写也是一些语言中非常重要的处理对象。对此，显然最常见的方法是str.lower()，此外，还有str.casefold()，即大小写折叠。这两个函数对绝大多数字符来说是等价的，但是对于极少数的特殊字符会得到不同的结果（准确来说，本书提到自Python3.4起，有116个字符的处理结果不一致）

上述处理实际上均只涉及到单字符，但是在一些应用场合中，对字符进行适当的转换也是非常重要的处理手段。例如对于搜索引擎，用户会希望使用不同语言表示的，具有相同语义的关键字会返回相似的检索结果；此外，字符的一些标注符号（重音符、下加符等）应当不太影响检索结果。前者的处理较为复杂，后者则仍在字符串规范化的范畴内。

本书提供了一个去除变音符号的算法。具体而言，首先将所有的组合字符解析为基字符和组合字符（NFD格式），然后滤除所有的组合字符，最后进行重组即可（NFC格式）

## 字符串排序

类似于字符串比较，字符串排序也受到各种非常规的字符串的困扰（包含非ASCII字符的字符串）。Python提供了locale.strxfrm函数，该函数用于将字符串转换为适合所在区域的形式。

locale.strxfrm函数需要和locale.setlocale等函数配合使用，说白了就是进行本地化，只不过是自动进行的。

除了内置的函数外，PyUCA实现了Python版的UCA，这个库也可用于进行排序

## 其他模块中的字符串和字节序列

除了直接对字符串进行处理外，大量其他库也会涉及到字符串的运用。本书以正则表达式和os库中对文件目录的处理为例，展示这些模块中的字符串和字节序列处理方法。

就目前而言，在我的日常项目中通常是使用纯ASCII编码的文本来规避字符串处理上的困难，一些不得不处理的乱码问题（例如matplotlib中文乱码）也通常能找到解决办法。因此这部分内容等到有需要时再阅读。

## 总结

1. Python强制区分文本字符串以及这些字符串的二进制形式下的表述
2. 编解码问题是处理文本绕不开的话题，Python对这一块内容也有相当多的支持
3. UnicodeEncodeError和UnicodeDecodeError是字符串编解码问题中需要重点关注的两个异常，Python也提供了对相关异常进行可配置的处理手段
4. 如何在不知晓编码方式的情况下对字节序列进行解码显然是一个非常有趣的问题。虽然该问题无法100%解决，Chardet包仍然提供了对一些常见编码的处理手段
5. 字符串的规范化是进行一些更复杂操作的前提，Python的unicodedata库提供了normalize方法来实现这一功能；对于字符串排序，Python则提供了locale.strxfrm函数，此外也可以使用PyUCA等库
6. 对字符串进行正确处理是功能正常运转的前提，各个库/模块或多或少均涉及到字符串的处理，具体的处理方法等到有需要的时候再深入探索
7. 人类使用文本，计算机使用字节序列 